In [1]:
%pylab inline
import os
import time
import pandas as pd
from shutil import copyfile, rmtree
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras import optimizers
import scipy.io
from PIL import Image
from sklearn import metrics, preprocessing

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
def dynamic_std_print(string_to_print):
    """
    Printing string on standard output and refreshing output line
    """
    sys.stdout.write('\r')
    sys.stdout.write(string_to_print)
    sys.stdout.flush()

In [3]:
# Set folder delimiter
if os.name == 'posix':
    fd = "/"
else:
    fd = "\\"

# 1) Get data

Gather our files paths

In [4]:
# Get local path
local_path = os.getcwd()
pictures_path = local_path + '/data/images/'

In [5]:
# List images folders
breeds_folders = [local_path + "/data/images/" + file_name for file_name in os.listdir(local_path + "/data/images") if os.path.isdir(local_path + "/data/images/" + file_name)]

In [6]:
# List breeds id and name
breeds_id = [breed_path.split("/")[-1].split("-")[0] for breed_path in breeds_folders]
breeds_name = ["-".join(breed_path.split("/")[-1].split("-")[1:]) for breed_path in breeds_folders]
breed_dict = dict(zip(breeds_id, breeds_name))
reverse_breed_dict = dict(zip(breeds_name, breeds_id))
n_breeds = len(breeds_name)

Use train and test sets provided with data

In [7]:
train_mat = scipy.io.loadmat(local_path + '/data/lists/train_list.mat')
test_mat = scipy.io.loadmat(local_path + '/data/lists/test_list.mat')

Convert matlab matrix to pandas dataframe

In [8]:
# For train set
train_df = pd.DataFrame(train_mat['labels'], columns=['label'])
train_df['file'] = [str(file_name[0].tolist()[0]) for file_name in train_mat['file_list']]
train_df['breed_id'] = train_df.file.apply(lambda x: x.split("/")[0].split("-")[0])
train_df['breed_name'] = train_df.breed_id.apply(lambda x: breed_dict[x])
train_df.label = train_df.label.apply(lambda x: x - 1)

# For test set
test_df = pd.DataFrame(test_mat['labels'], columns=['label'])
test_df['file'] = [str(file_name[0].tolist()[0]) for file_name in test_mat['file_list']]
test_df['breed_id'] = test_df.file.apply(lambda x: x.split("/")[0].split("-")[0])
test_df['breed_name'] = test_df.breed_id.apply(lambda x: breed_dict[x])
test_df.label = test_df.label.apply(lambda x: x - 1)

In [9]:
# Get label-names dict
breed_labels_df = train_df.loc[:, ['label', 'breed_name']].drop_duplicates()
breed_label_dict = dict(zip(breed_labels_df.label.values, breed_labels_df.breed_name.values))
label_breed_dict = dict([[v,k] for k,v in breed_label_dict.items()])

In [10]:
train_df.sample(5)

,label,file,breed_id,breed_name
10701,107,n02112018-Pomeranian/n02112018_9376.jpg,n02112018,Pomeranian
3552,35,n02094258-Norwich_terrier/n02094258_905.jpg,n02094258,Norwich_terrier
6633,66,n02102040-English_springer/n02102040_3335.jpg,n02102040,English_springer
2169,21,n02091134-whippet/n02091134_11775.jpg,n02091134,whippet
6948,69,n02102480-Sussex_spaniel/n02102480_5805.jpg,n02102480,Sussex_spaniel


Copy files to train and test folders for CNN training

In [38]:
# # Copy train files to train folder
# for file in train_df.file.values:
#     folder = file.split("/")[0]
#     file_name = file.split("/")[-1]
#     if not os.path.exists(local_path + '\\data\\train\\' + folder):
#         os.mkdir(local_path + '\\data\\train\\' + folder)
#         dynamic_std_print("Creating folder {}                  ".format(folder))
#     copyfile(pictures_path + file, local_path + '\\data\\train\\' + folder + '\\' + file_name)
    
# # Copy test files to test folder
# for file in test_df.file.values:
#     folder = file.split("/")[0]
#     file_name = file.split("/")[-1]
#     if not os.path.exists(local_path + '\\data\\test\\' + folder):
#         os.mkdir(local_path + '\\data\\test\\' + folder)
#         dynamic_std_print("Creating folder {}                  ".format(folder))
#     copyfile(pictures_path + file, local_path + '\\data\\test\\' + folder + '\\' + file_name)

#### Copy files to small train and test folders for CNN training (few files for debuging purposes)

In [11]:
folder_name = "cnn2"

Remove files

In [76]:
# Remove folders content
folders = [local_path + fd + 'data' + fd + folder_name + fd + 'train' + fd,
           local_path + fd + 'data' + fd + folder_name + fd + 'test' + fd + 'test_folder',
           local_path + fd + 'data' + fd + folder_name + fd + 'valid' + fd,
           local_path + fd + 'data' + fd + folder_name + fd + 'test_folder' + fd]
for folder in folders:
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                rmtree(file_path)
        except Exception as e:
            print(e)

Set list of breeds matching histograms + kmeans method for comparison purpose

In [12]:
list_labels = ['Scotch_terrier', 'Eskimo_dog', 'Rhodesian_ridgeback', 'Great_Dane', 'French_bulldog']

#### Extract specific pictures for CNN usage

In [79]:
# Copy train and valid  files to train and valid folders
# Get n pic for each label
n_train = 90
n_valid = 10
n_test = 30

for label_name in list_labels: # train_df.label.unique():
    # Convert dog's name to label number
    label = label_breed_dict[label_name]
    # Slice train DF to specified label
    label_df = train_df[train_df.label == label].sample(n_train + n_valid, random_state=0)
    
    # Copy training files
    for file in label_df.file.values[:n_train]:
        folder = file.split("/")[0]
        file_name = file.split("/")[-1]
        if not os.path.exists(local_path + fd + 'data' + fd + folder_name + fd + 'train' + fd + folder):
            os.mkdir(local_path + fd + 'data' + fd + folder_name + fd + 'train' + fd + folder)
            dynamic_std_print("Creating folder {}                             ".format(folder))
        copyfile(pictures_path + file, local_path + fd + 'data' + fd + folder_name + fd + 'train' + fd + folder + fd + '' + file_name)
        
    # Copy validation files        
    for file in label_df.file.values[n_train:]:
        folder = file.split("/")[0]
        file_name = file.split("/")[-1]
        if not os.path.exists(local_path + fd + 'data' + fd + folder_name + fd + 'valid' + fd + folder):
            os.mkdir(local_path + fd + 'data' + fd + folder_name + fd + 'valid' + fd + folder)
            dynamic_std_print("Creating folder {}                             ".format(folder))
        copyfile(pictures_path + file, local_path + fd + 'data' + fd + folder_name + fd + 'valid' + fd + folder + fd + '' + file_name)

    label_df = test_df[test_df.label == label].sample(n_test, random_state=0)
    # Copy testing files        
    for file in label_df.file.values:
        folder = file.split("/")[0]
        file_name = file.split("/")[-1]
        if not os.path.exists(local_path + fd + 'data' + fd + folder_name + fd + 'test_folder' + fd + folder):
            os.mkdir(local_path + fd + 'data' + fd + folder_name + fd + 'test_folder' + fd + folder)
            dynamic_std_print("Creating folder {}                             ".format(folder))
        copyfile(pictures_path + file, local_path + fd + 'data' + fd + folder_name + fd + 'test_folder' + fd + folder + fd + '' + file_name)

# Copy test files to test folder
# Get m pic for each label
target_names = []
for label_name in list_labels: #train_df.label.unique():
    # Convert dog's name to label number
    label = label_breed_dict[label_name]
    # Slice test DF to specified label
    label_df = test_df[test_df.label == label].sample(n_test, random_state=0)
    for file in label_df.file.values:
        folder = file.split("/")[0]
        file_name = file.split("/")[-1]
        file_breed = file_name.split("_")[0]
        target_names.append(file_breed)
        copyfile(pictures_path + file, local_path + fd + 'data' + fd + folder_name + fd + 'test' + fd + 'test_folder' + fd + file_name)

Creating folder n02108915-French_bulldog                                  

Build new labels list from sub-folders

In [13]:
cnn_targets = []
for subfold in os.listdir(local_path + fd + 'data' + fd + folder_name + fd + 'train'):
    cnn_targets.append(subfold.split("-")[1])
print("Here are the breeds in the training folder of our CNN : \n", cnn_targets)
# Create test pictures list
test_pictures_breeds = []
for i in range(len(list_labels)):
    for j in range(n_test):
        test_pictures_breeds.append(cnn_targets[i])

Here are the breeds in the training folder of our CNN : 
 ['Rhodesian_ridgeback', 'Scotch_terrier', 'French_bulldog', 'Great_Dane', 'Eskimo_dog']


NameError: name 'n_test' is not defined

Associate local labels to those breeds folders

In [14]:
# Create label encoder
subfold_enc = preprocessing.LabelEncoder()
# Fit encoder on breeds names
subfold_enc.fit(cnn_targets)
# Convert breeds names
cnn_targets_labels = subfold_enc.transform(cnn_targets)
print("Here are the encoded breeds in the training folder of our CNN : \n", cnn_targets_labels)

Here are the encoded breeds in the training folder of our CNN : 
 [3 4 1 2 0]


# 2) Modify Conventional Neural Network for Transfer Learning

We will use Keras VGG-16 CNN already trained on ImageNet database in order to predict our dogs breeds.

Steps are as follow :
- 1) Call the pre-trained VGG-16 CNN
- 2) Apply partial fine-tuning because we don't have much data (20e3) and they are dogs only, unlike ImageNet database
- 3) Run modified model for dogs classification

#### Prepare pictures batches

Training batch

In [15]:
train_batch_size = 90
# Initialise data generator
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# Fit generator from train folder. Folder must contains one sub-directory by class
train_generator = train_datagen.flow_from_directory(local_path + '/data/' + folder_name + '/train',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=train_batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 450 images belonging to 5 classes.


Validation batch

In [16]:
valid_batch_size = 10
# Initialise data generator
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# Fit generator from train folder. Folder must contains one sub-directory by class
valid_generator = valid_datagen.flow_from_directory(local_path + '/data/' + folder_name + '/valid',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=valid_batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 50 images belonging to 5 classes.


Test batch : from test sub-folders with anotations

In [17]:
test_batch_size = 30
# Initialise data generator
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# Fit generator from train folder. Folder must contains one sub-directory by class
test_generator = test_datagen.flow_from_directory(local_path + '/data/' + folder_name + '/test_folder',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=test_batch_size,
                                                  class_mode='categorical',
                                                  shuffle=True)

Found 150 images belonging to 5 classes.


Test batch : from a unique test sub-folder

In [18]:
test_batch_size = 10
# Initialise data generator
test_datagen_unique = ImageDataGenerator(preprocessing_function=preprocess_input)
# Fit generator from train folder. Folder must contains one sub-directory by class
test_gen_unique = test_datagen_unique.flow_from_directory(local_path + '/data/' + folder_name + '/test',
                                                         target_size=(224,224),
                                                         color_mode='rgb',
                                                         batch_size=test_batch_size,
                                                         class_mode='categorical',
                                                         shuffle=True)

Found 150 images belonging to 1 classes.


In [19]:
# # Test on generator content
# count=1
# for item in test_generator:
#     plt.imshow(item[0][0])
#     break
# #     print(count)
# #     save_test_item = item
# #     break
# #     count+=1

#### Prep model

In [20]:
# Load VGG-16 pre-trained on ImageNet without the fully-connected layers
model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Get output of this model
x = model.output

# ???
x = model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# let's add two fully-connected layer
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Add new fully-connected layer for N classes classification
predictions = Dense(5, activation='softmax')(x)

# Define new model
new_model = Model(inputs=model.input, outputs=predictions)

In [21]:
# Do not train 5 first layers (the lowest)
for layer in new_model.layers[:5]:
   layer.trainable = False

In [22]:
# Compile model
new_model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

What's in our model so far ?

In [23]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [30]:
# train on training generator
n_epochs = 100
model_complete = new_model.fit_generator(train_generator,
                                         steps_per_epoch=10,
                                         epochs=n_epochs,
                                         validation_data=valid_generator,
                                         validation_steps=10,
                                         verbose=1)

Epoch 1/100
 3/10 [========>.....................] - ETA: 8:28 - loss: 0.0320 - acc: 0.9926

KeyboardInterrupt: 

Epochs accuracy evolution ploting

In [ ]:
# Plot training & validation accuracy values
plt.plot(model_complete.history['acc'])
plt.plot(model_complete.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(model_complete.history['loss'])
plt.plot(model_complete.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

Evaluate test generator to get model accuracy

In [ ]:
new_model.evaluate_generator(test_generator, steps=10)

Predict on test pictures

In [ ]:
pred = new_model.predict_generator(test_generator, steps=10, verbose=1)

Retrieve dictionnary of classes of test set

In [71]:
test_map = dict((v, k.split("-")[1]) for k, v in test_generator.class_indices.items())
test_map

{0: 'Rhodesian_ridgeback',
 1: 'Scotch_terrier',
 2: 'French_bulldog',
 3: 'Great_Dane',
 4: 'Eskimo_dog'}

#### Get max proba for each picture (most probable class i.e. dog's breed)

### The following accuracy is not to be trusted as representative of the model because we don't know if the model is predicting on test pictures in the same order as these pictures are ordered in the test directory

In [72]:
# get max proba
max_preds = pred.argmax(axis=1)
print(max_preds)
# convert to dataframe
pred_df = pd.DataFrame(max_preds, columns=['pred_label'])
pred_df['pred_name'] = pred_df.pred_label.apply(lambda x: test_map[x])
pred_df['true_name'] = test_pictures_breeds
pred_df['true_id'] = pred_df.true_name.apply(lambda x: reverse_breed_dict[x])

# pred_df['true_id'] = target_names
# pred_df['true_name'] = pred_df.true_id.apply(lambda x: breed_dict[x])
# pred_df['true_label'] = pred_df.true_name.apply(lambda x: label_breed_dict[x])
pred_df['true_label'] = pred_df.true_name.apply(lambda x: subfold_enc.transform([x])[0])
pred_df['accurate'] = pd.Series([True if predict == truth else False for predict, truth in pred_df.loc[:, ['pred_label', 'true_label']].values])

[0 0 0 0 0 0 0 0 0 0]


In [73]:
pred_df

,pred_label,pred_name,true_name,true_id,true_label,accurate
0,0,Rhodesian_ridgeback,Rhodesian_ridgeback,n02087394,3,False
1,0,Rhodesian_ridgeback,Rhodesian_ridgeback,n02087394,3,False
2,0,Rhodesian_ridgeback,Scotch_terrier,n02097298,4,False
3,0,Rhodesian_ridgeback,Scotch_terrier,n02097298,4,False
4,0,Rhodesian_ridgeback,French_bulldog,n02108915,1,False
5,0,Rhodesian_ridgeback,French_bulldog,n02108915,1,False
6,0,Rhodesian_ridgeback,Great_Dane,n02109047,2,False
7,0,Rhodesian_ridgeback,Great_Dane,n02109047,2,False
8,0,Rhodesian_ridgeback,Eskimo_dog,n02109961,0,True
9,0,Rhodesian_ridgeback,Eskimo_dog,n02109961,0,True


Get accuracy

In [74]:
metrics.accuracy_score(pred_df.true_label, pred_df.pred_label)

0.2

Train dummy classifier for comparison

In [75]:
target_testing = pred_df.true_label.values
mean_rand = []
for i in range(5000):
    random_pred = []
    for target in target_testing :
        random_pred.append(np.random.randint(target_testing.min(), target_testing.max()))
    random_pred = np.array(random_pred)
    mean_rand.append(np.equal(target_testing.reshape(-1), random_pred).sum() / len(target_testing))
mean(mean_rand)

0.20142000000000002